# Enterprise Excel Writer: Full Feature Demo

This notebook demonstrates the complete capability set of the `EnterpriseExcelWriter`. It creates a complex, multi-tab executive report showing off layout control, advanced formatting, visuals, and multiple ways to handle data dictionaries.

### Prerequisites
Ensure your `src` folder is populated and your `tests/data.db` is set up (run `tests/update_db_gary_test.py` to ensure the 'GaryTest' profile exists).

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime

# 1. Setup Path to Source
# Assumes notebook is in 'notebooks/' and source is in 'src/'
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from enterprise_writer import EnterpriseExcelWriter
from config_provider import fGetReportConfig

print("Libraries Loaded.")

Libraries Loaded.


### 1. Data Preparation
We create mock data that includes specific features to test:
- **Dates:** To test auto-formatting.
- **URLs:** To test hyperlink detection.
- **Custom Units:** To test format overrides (e.g., 'kg').
- **Rich Text:** For the advanced data dictionary example.

In [2]:
# Mock Sales Data
data_sales = [
    ['North', datetime.date(2025, 1, 15), 1500.50, 0.98, 'https://fabric.microsoft.com', 500],
    ['South', datetime.date(2025, 1, 16), 2300.00, 0.45, 'https://www.python.org', 750],
    ['East',  datetime.date(2025, 1, 17), 1200.25, 0.88, 'mailto:support@company.com', 300],
    ['West',  datetime.date(2025, 1, 18), 3100.00, 0.92, 'https://seaborn.pydata.org', 900]
]
dfSales = pd.DataFrame(data_sales, columns=['region_name', 'run_date', 'revenue', 'efficiency', 'link', 'weight_kg'])

# Mock Data Dictionary
# Note the 'excel_format' column for custom overrides
data_dict = [
    ['region_name', 'Operating Region', None, 'Geographic location.'],
    ['run_date', 'Report Date', 'dd-mmm-yy', 'Date of extract.'],
    ['revenue', 'Total Revenue', '£#,##0', 'Gross revenue pre-tax.'],
    ['efficiency', 'Eff. Score', '0.0%','Operational efficiency.'],
    ['link', 'Source System', None, 'Direct link to source record.'],
    ['weight_kg', 'Total Weight', '#,##0 "kg"', 'Total shipping weight.']
]
dfDict = pd.DataFrame(data_dict, columns=['column_name', 'display_name', 'excel_format', 'description'])

print("Data Ready.")

Data Ready.


### 2. Initialization
We load the `GaryTest` profile. This pulls in the Indigo/Lavender colour scheme, font sizes, and logo paths from the database.

In [3]:
# Fetch Configuration
vConfig = fGetReportConfig('GaryTest')

# Initialize Writer
vReport = EnterpriseExcelWriter("Full_Feature_Report.xlsx", vConfig=vConfig)

# Apply Column Mappings (Technical -> Display Names & Custom Formats)
vReport.fSetColumnMapping(dfDict)

### 3. Tab 1: Visual Dashboard
This tab demonstrates **Pixel Perfect Layout**.
1.  **Banner:** Full-width "Confidential" strip.
2.  **Logo & Title:** Standard branding.
3.  **Seaborn Chart:** A Python-generated image chart inserted at the top.
4.  **Rich Text:** Adding a styled note.

In [4]:
# 1. Sensitivity Banner (Uses 'Warning' config: Blue BG, White Text)
vReport.fAddBanner("OFFICIAL - SENSITIVE - INTERNAL USE ONLY", "Warning")

# 2. Logo & Title
vReport.fAddLogo(vPos="A3") # Manually place logo at A3 (below banner)
vReport.fSkipRows(1) # Move cursor down past logo space
vReport.fAddTitle("Executive Performance Dashboard")

# 3. KPI Cards
vReport.fAddKpiRow({'Total Revenue': '£8.1k', 'Avg Efficiency': '81%', 'Active Regions': '4'})

# 4. Seaborn Chart (Image)
# We generate a Python chart and insert it. The writer handles the theming.
vReport.fAddSeabornChart(
    dfSales, 
    vXCol='region_name', 
    vYCol='revenue', 
    vTitle='Revenue by Region (Seaborn)', 
    vChartType='bar'
)

# 5. Rich Text Note
vReport.fAddText([
    "Note: The region ", 
    {'text': 'South', 'bold': True, 'colour': 'red'}, 
    " is underperforming this quarter."
])

### 4. Tab 2: Detailed Data
This tab focuses on **Table Features**.
1.  **Freeze Panes:** Locks the header row.
2.  **Manual Widths:** Forces the 'Link' column to be wide.
3.  **Auto-Filter:** Adds dropdowns to headers.
4.  **Totals:** Writes `=SUM()` formulas at the bottom.
5.  **Native Chart:** Adds an interactive Excel chart linked to the data.

In [5]:
vReport.fNewSheet("Details", "Full breakdown of regional metrics")

vReport.fAddTitle("Regional Data Breakdown")

# 1. Freeze Panes (Row 3, Col 0 - Header is at Row 3)
vReport.fFreezePanes(3, 0)

# 2. Manual Column Widths (Region=15, Link=40)
vReport.fSetColumnWidths({0: 15, 4: 40})

# 3. Write Data (With AutoFilter and Totals)
# Note: Hyperlinks in 'link' and Dates in 'run_date' are auto-detected
vReport.fWriteDataframe(dfSales, vAddTotals=True, vAutoFilter=True)

# 4. Conditional Formatting
# Highlight Efficiency < 0.50 (50%)
vReport.fAddConditionalFormat('efficiency', 'cell', {'criteria': '<', 'value': 0.50}, vColour='#FFC7CE')

# 5. Native Excel Chart (Linked)
vReport.fAddChart(
    vTitle="Revenue (Interactive Excel Chart)",
    vType="column",
    vXAxisCol="region_name",
    vYAxisCols=["revenue"]
)

### 5. Tab 3: Data Dictionary (The 3 Methods)
Here we demonstrate the three specific methods for adding reference data requested.

1.  **Standard Table:** Auto-filtered to show only used columns.
2.  **Rich Text Table:** Allows mixing bold/colors inside cells.
3.  **Definition List:** The "Guidance Notes" style (Bold Key: Italic Value).

In [6]:
vReport.fNewSheet("Appendix", "Data Definitions")

# --- METHOD 1: Standard Dictionary (Auto-Filtered) ---
vReport.fAddTitle("Method 1: Standard Table (Auto-Filtered)", vFontSize=12)
vReport.fAddText("This table automatically filters to show only columns present in the report.")
# Note: fAddDataDictionary implicitly calls fFilterDataDictionary
vReport.fAddDataDictionary(dfDict)

vReport.fSkipRows(2)

# --- METHOD 2: Rich Text Dataframe ---
vReport.fAddTitle("Method 2: Rich Text Table", vFontSize=12)

# A. Manually Filter
dfFiltered = vReport.fFilterDataDictionary(dfDict)

# B. Create a 'Rich' version (Modify a cell to be a list)
dfRich = dfFiltered.copy()
# Example: Make the 'Region' description have RED text
dfRich.loc[0, 'description'] = ["Geographic ", {'text': 'operational', 'colour': 'red', 'bold': True}, " area."]

# C. Write using the Rich Writer
vReport.fWriteRichDataframe(dfRich)

vReport.fSkipRows(2)

# --- METHOD 3: Definition List (Guidance Style) ---
vReport.fAddTitle("Method 3: Guidance Notes Style", vFontSize=12)

# A. Prepare Data (Must be 2 columns: Term, Definition)
# We select just the Display Name and Description from our filtered dict
dfDefinitions = dfFiltered[['display_name', 'description']]

# B. Add the list
vReport.fAddDefinitionList(dfDefinitions)

### 6. Finish
Generate the Table of Contents and save the file.

In [7]:
vReport.fGenerateTOC()
vReport.fClose()

File saved: Full_Feature_Report.xlsx
